In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
%matplotlib inline
import csv
import sys

In [73]:
from aqi import aqi_year
from bs4 import BeautifulSoup

In [74]:
file_html = open('Html_data/2013/1.html', 'rb')
file_html=file_html.read()

In [75]:
soup  = BeautifulSoup(file_html, 'lxml')

In [76]:
a = soup.find_all('table', {'class' : 'medias mensuales numspan'})

In [79]:
def met_data(month, year):
#     Accessing file stored in html_data folder in format year/month wise
    file_html = open('Html_Data/{}/{}.html'.format(year,month), 'rb') 
#     reading the binary format data 
    plain_text = file_html.read()

    tempD = [] # This is our list with all the values of every column
    finalD = [] # This our our list which will contain every row

#     Parsing data in lxml format using  Beautiful Soup
    soup = BeautifulSoup(plain_text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}): #There is only one table but for being on a safe side using this command
        for tbody in table: #Accesing every tbody
            for tr in tbody: #Accesing every row of table
                a = tr.get_text() # getting text out of the row
                tempD.append(a) #inserting into a list

    rows = len(tempD) / 15   #There are total 15 Columns in the table so after every 15 number new columns is starting

#   For every row it will iterate for 15 times as total columns are 15
    for times in range(round(rows)): 
        newtempD = [] # This will be out column list
        for i in range(15):
            newtempD.append(tempD[0]) # Appending first value of our main list to column list
            tempD.pop(0) # Deleting the first value so in next iteration it will not get append again
        finalD.append(newtempD) # Appending every column list to our final row list

    length = len(finalD) # Checing total number of rows in our row list

    # Deleting out rows
    finalD.pop(length - 1) 
    finalD.pop(0)

#     Deleting all useless columns
    for a in range(len(finalD)):
        finalD[a].pop(6)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(9)
        finalD[a].pop(0)

# This is our final filtered table  
    return finalD

# This function is to get list of data year wise so at last we can make a final dataframe of all years
def data_combine(year, cs):
    for a in pd.read_csv('RealData/real_' + str(year) + '.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        df.drop('SLP', axis=1, inplace=True)
        mylist = df.values.tolist()
    return mylist


if __name__ == '__main__':
#     Creating a directory to store real data
    if not os.path.exists('RealData'):
        os.makedirs('RealData')
        
#         Extracting data year wise
    for year in range(2013,2019):
        final_data = []
#         creating a csv file to store heading row year wise
        with open('RealData/real_' + str(year) + '.csv', 'w') as csvfile:
            wr = csv.writer(csvfile, dialect = 'excel')
            wr.writerow(['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM2.5'])
#             exctracting data month wise from our met_data function
        for month in range(1,13):
            temp = met_data(month, year)
            final_data = final_data + temp
            
#       Now getting AQI data from the function created in AQI.py file... CHECK THAT OUT FOR DETAIL      
        pm = aqi_year(str(year))
        
        if len(pm)==364:
            pm.insert(364, '-')
            
#         Inserting AQI to last column of final_data
        for i in range(len(final_data)-1):
            final_data[i].insert(8, pm[i])
            
#       Writing final data list to our csv file created before
        with open('RealData/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect = 'excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem =='':
                        flag=1
                if flag!=1:
                    wr.writerow(row)
#   Getting dataframe list from data_combine function so later on data can be combined to one dataframe
    data_2013 = data_combine(2013, 600)
    data_2014 = data_combine(2014, 600)    
    data_2015 = data_combine(2015, 600)    
    data_2016 = data_combine(2016, 600)  
    data_2017 = data_combine(2017, 600)  
    data_2018 = data_combine(2018, 600)  

#     Combining list to one list named as total
    total = data_2013+data_2014+data_2015+data_2016+data_2017+data_2018
    
# adding all year data to one csv file
    with open('RealData/Real_combine.csv', 'w') as csvfile:
        wr = csv.writer(csvfile, dialect = 'excel')
        wr.writerow(['T', 'TM', 'Tm', 'H', 'VV', 'V', 'VM', 'PM2.5'])
        wr.writerows(total)

In [95]:
df = pd.read_csv('RealData/Real_combine.csv')

In [97]:
df.tail()

,T,TM,Tm,H,VV,V,VM,PM2.5
1123,22.7,28.4,18.1,65,6.3,3.9,5.4,310.250000
1124,23.4,29.1,18.4,62,6,5.7,9.4,204.375000
1125,23.8,30.2,17.7,56,5.5,6.1,9.4,223.250000
1126,24,31.4,19,63,6.3,5.4,7.6,248.958333
1127,20.9,25.8,16.6,51,6.6,3.9,11.1,NaN


In [100]:
len(aqi_year('2018'))

364

In [102]:
aqi_year('2018')[363]

334.875

In [104]:
df[df['PM2.5']==310.250000]

,T,TM,Tm,H,VV,V,VM,PM2.5
369,21.1,28.3,17.1,74,7.7,0.9,3.5,310.25
1123,22.7,28.4,18.1,65,6.3,3.9,5.4,310.25
